In [1]:
import os
import sys
import datetime
import pprint

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/crewAI/..


In [2]:
from models.llm import LLM

model = LLM('gpt-4o-mini')
llm = model.load_model()
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10cf6f1f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10cf98b50>, root_client=<openai.lib.azure.AzureOpenAI object at 0x109ed5990>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x10cf6f220>, temperature=0.0, max_tokens=4096, deployment_name='gpt-4o-mini', openai_api_version='2023-05-15')

In [3]:
os.environ["OTEL_SDK_DISABLED"] = "true"
os.environ["CREWAI_TELEMETRY_OPT_OUT"] = "true"

### Search Tools

#### Tavily Search

1000 search credit / month

In [4]:
from langchain_community.tools import TavilySearchResults

tavily_search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=False,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)


In [5]:
results = tavily_search.invoke({"query": "What happened with the SENSEX yesterday?"})
pprint.pp(results)

[{'url': 'https://www.hindustantimes.com/business/stock-market-crash-today-sensex-nifty-us-federal-rate-cut-101734578317776.html',
  'content': 'Market yesterday Stock market crash today: On Wednesday, the '
             'Sensex closed at 80,182.20 points, down 502.25 points or 0.62 '
             'per cent, while the Nifty ended at 24,198.85 points, declining '
             'by 137.15'},
 {'url': 'https://economictimes.indiatimes.com/markets/stocks/news/rs-6-lakh-crore-wiped-out-as-sensex-nifty-slide-1-each-key-factors-behind-todays-market-crash/articleshow/113893073.cms',
  'content': 'Stock Market Crash Today: Sensex fell steeply by over 1,750 '
             'points and Nifty50 dropped to 25,250 due to heightened Middle '
             'East tensions. Iran launched missiles at Israel, causing '
             "investor worry. Rising oil prices and SEBI's F&O measures also "
             'impacted market sentiment. Concern over resurgent Chinese stocks '
             'led to significant

#### Serper Search

Total: 2500 search credit free

In [6]:
from langchain_community.utilities import GoogleSerperAPIWrapper

# serper_search = GoogleSerperAPIWrapper()

In [7]:
# results = serper_search.results("Studies on cardiovascular impacts of weight loss drug from Lilly?")


### Crew with single agent and single task

In [8]:
from crewai import Agent, Task, Crew

# Instantiate tools
search_tool = tavily_search

# Create agents
research_agent = Agent(
  role='Researcher',
  goal='Find and summarize the latest AI news',
  backstory="""You're a researcher at a large company.
  You're responsible for analyzing data and providing insights
  to the business.""",
  llm=llm,
  tools=[search_tool],
  verbose=True
)

# Define tasks
task = Task(
  description='Find and summarize the latest AI news',
  expected_output='A bullet list summary of the top 5 most important AI news',
  agent=research_agent,
)

# Assemble a crew
crew = Crew(
    agents=[research_agent],
    tasks=[task],
    verbose=True,
    iterations=3
)

# Execute tasks
result = crew.kickoff()
print(result)

2025-01-10 14:14:18,376 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.
2025-01-10 14:14:18,381 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.
2025-01-10 14:14:18,384 - 8638448192 - manager.py-manager:293 - WARNING: Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 [2025-01-10 14:14:18][DEBUG]: == Working Agent: Researcher
 [2025-01-10 14:14:18][INFO]: == Starting Task: Find and summarize the latest AI news
I need to gather the latest AI news to provide a comprehensive summary. I'll use the search tool to find the most recent and relevant information.

Action: tavily_search_results_json  
Action Input: {"query":"latest AI news"}  


2025-01-10 14:14:27,924 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.


 

[{'url': 'https://opentools.ai/news', 'content': "Latest AI News: September 6, 2024 Australia Takes the Lead in AI Regulation with New Human Oversight Rules Australia's government announces plans to introduce specific AI rules focusing on human oversight an... OpenAI Co-Founder Sutskever's New AI Safety Startup SSI Secures $1 Billion in Funding! Amazon Bolsters AI Robotics by Hiring Covariant's Founders Amazon's latest move involves onboarding the founders of Covariant, a startup specializing in AI-dri... Stability AI Expands Horizons with New Models on Amazon Bedrock AI Set to Shake Up Oil Prices Over the Next Decade, Says Goldman Sachs Join 47,000+  learning AI in 3 minutes daily Join our free daily newsletter talking about AI News, AI Products, AI tutorials, and more."}, {'url': 'https://www.sciencedaily.com/news/computers_math/artificial_intelligence/', 'content': "Nov. 12, 2024 — Researchers have developed PanoRadar, a new tool to give robots superhuman vision by transforming s

#### Crew With Planning

In [12]:
import os
from crewai import Agent, Task, Crew

# Instantiate tools
search_tool = tavily_search

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI in Healthcare industry and AI in Finance industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool],
    llm=llm,
    verbose=True
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI in Healthcare industry and AI in Finance industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments with a unique perspective on their significance.',
    agent=researcher
)


# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher],
    tasks=[research],
    llm=llm,
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute tasks
result = crew.kickoff()

print(result)

2025-01-10 14:16:29,214 - 8638448192 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
2025-01-10 14:16:29,216 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.


2025-01-10 14:16:29,239 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.


 [2025-01-10 14:16:29][INFO]: Planning the crew execution
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.


2025-01-10 14:16:47,997 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.
2025-01-10 14:16:48,004 - 8638448192 - manager.py-manager:293 - WARNING: Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 [2025-01-10 14:16:47][DEBUG]: == Working Agent: Market Research Analyst
 [2025-01-10 14:16:47][INFO]: == Starting Task: Research the latest trends in the AI in Healthcare industry and AI in Finance industry and provide a summary.1. Define Key Areas of Focus: Identify specific applications such as diagnostics, patient management systems, predictive analytics, and personalized medicine. 2. Utilize Research Tools: Use TavilySearchResults with settings including raw content and images. 3. Conduct Searches: Input queries like 'latest trends in AI in healthcare 2023', review and bookmark relevant articles and reports. 4. Analyze Collected Data: Skim through articles to identify top developments based on relevance and potential impact. 5. Draft the Summary: Structure the summary with trend title, description, and significance for each of the top 3 trends. 6. Review and Refine: Edit for clarity and coherence, incorporating visuals as needed. 7. Prepare for Presentation: Finalize document form

2025-01-10 14:17:00,910 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.


 

[{'url': 'https://www.ibef.org/blogs/the-future-of-artificial-intelligence-in-healthcare-transforming-patient-care', 'content': "These trends collectively illustrate AI's significant contribution to enhancing healthcare delivery, efficiency, and patient outcomes in 2023. This exciting new phase in healthcare sets the stage for a doctor-patient relationship with AI serving as an insightful partner on the journey to improved health. AI plays a role in this evolution, enabling the monitoring of patients from a distance and introducing an era of proactive healthcare. From revolutionising disease detection to empowering remote patient monitoring, AI is transforming healthcare at an unprecedented pace. Personalised healthcare, made possible by AI's capacity to analyse patient information, is paving the way for tailored treatment plans that cater to each patient's requirements. It is crucial to ensure the development and implementation of AI in healthcare to uphold fairness, and transparen

2025-01-10 14:17:09,425 - 8638448192 - __init__.py-__init__:1231 - WARNING: SDK is disabled.


 

[{'url': 'https://stayrelevant.globant.com/en/technology/finance/ai-financial-services-2023-highlights-trends-insights-recap/', 'content': 'In the episode, AI in the world of finance, Lander Gamboa, Director of the Finance Studio at Globant, Rachel Armstrong, Studio Partner at Globant UK, and Carlos Gonzalez, partner of the Finance Studio at Globant, discuss how AI is opening the path for new solutions and bringing up critical challenges for financial institutions.\n As part of our Applied AI efforts at Globant, we launched the Tech Talks series, including The Next Era of Finance: Harnessing the Power of AI, where we take the conversation of the transformative power of AI in the banking arena further.\n With AI taking center stage in 2023, we delved into how this technology is already changing the banking and financial services landscape in Globant’s report, Applied AI in Financial Services: A New Banking Era.\nSpecialists from Globant’s Finance Studio share how the financial sector

In [13]:
print(result)

### Summary of Top Trends in AI in Healthcare and Finance for 2023

#### AI in Healthcare

1. **Personalized Medicine and Predictive Analytics**
   - AI is enabling healthcare providers to create tailored treatment plans and forecast patient outcomes, enhancing patient care and resource efficiency.

2. **Remote Patient Monitoring and Telehealth**
   - The rise of remote monitoring technologies allows for real-time health tracking, improving access to care and proactive management of chronic conditions.

3. **Natural Language Processing (NLP) and Conversational AI**
   - Advances in NLP are transforming patient interactions, streamlining administrative processes, and enhancing patient engagement through virtual assistants.

#### AI in Finance

1. **Generative AI for Enhanced Decision-Making**
   - Financial institutions are adopting generative AI tools to improve decision-making, automate reports, and enhance productivity, leading to better customer service.

2. **Hyper-Personalization 